In [1]:
import pandas as pd
import numpy as np
import pickle as pkl

movie_db = pd.read_pickle("C:/Users/soura/RL_Dialogue_Systems/data/movie_db.pkl")
movie_dict = pd.read_pickle('data/movie_dict.pkl')
movie_user_goals = pd.read_pickle('data/movie_user_goals.pkl')

In [2]:
  
from db_query import DBQuery
import numpy as np
from utils import convert_list_to_dict
from dialogue_config import all_intents, all_slots, usersim_default_key
import copy
import pickle, argparse, json, math

### Testing DB Queries

In [3]:
#Load file path constants
CONSTANTS_FILE_PATH = 'constants.json'
constants_file = CONSTANTS_FILE_PATH
with open(constants_file) as f:
        constants = json.load(f)
CONSTANTS_FILE_PATH = 'constants.json'
file_path_dict = constants['db_file_paths']
DATABASE_FILE_PATH = file_path_dict['database']
DICT_FILE_PATH = file_path_dict['dict']
USER_GOALS_FILE_PATH = file_path_dict['user_goals']

In [4]:
database = pickle.load(open(DATABASE_FILE_PATH, 'rb'), encoding='latin1')

In [5]:
db_helper = DBQuery(database)

Returns the value of the number of occurrence of the inform slots given a key, this will be further used by the “Fill information slot” method to retrieve the maximum count of the keys retrieved from DB.

In [6]:
current_informs = {'moviename':'the witch','city':'seattle'}

In [7]:
db_results_dict = db_helper.get_db_results_for_slots(current_informs)

In [8]:
db_results_dict

{'moviename': 41, 'city': 303, 'matching_all_constraints': 6}

This function is meant to fill in the required information from the database. It searches through the database to fill the inform slots PLACEHOLDER with values that work given the constraints of the current episode. This function also makes use of 2 other methods one to retrieve DB results for constraints and another to get the maximum number of the filled-in information ii.e. slot values with the highest count of available results which are described in the previous sections.

In [9]:
agent_inform = {'theater':''}
current_informs = {'moviename':'the witch','city':'seattle'}

In [10]:
inform_slots = db_helper.fill_inform_slot(agent_inform, current_informs)

In [11]:
inform_slots

{'theater': 'regal meridian 16'}

This function takes in input all the constraints information in a conversation and matches with the database if all the constraints are satisfied, and their values match then the item  is added to the returned dictionary

In [12]:
current_informs = {'moviename':'the witch','city':'seattle'}

In [13]:
db_results = db_helper.get_db_results(current_informs)

In [14]:
db_results

{49: {'date': 'tomorrow',
  'city': 'seattle',
  'theater': 'regal meridian 16',
  'moviename': 'the witch',
  'starttime': '9:30 pm'},
 66: {'date': 'tomorrow',
  'city': 'seattle',
  'theater': 'regal meridian 16',
  'moviename': 'the witch',
  'starttime': '9:30 pm'},
 147: {'date': 'tomorrow',
  'city': 'seattle',
  'theater': 'regal meridian 16',
  'moviename': 'the witch',
  'starttime': '9:30 pm'},
 810: {'date': 'tomorrow',
  'city': 'seattle',
  'theater': 'regal meridian 16',
  'moviename': 'the witch',
  'starttime': '9:30 pm'},
 850: {'date': 'tomorrow',
  'city': 'seattle',
  'theater': 'regal meridian 16',
  'moviename': 'the witch',
  'starttime': '9:30 pm'},
 851: {'date': 'tomorrow',
  'city': 'seattle',
  'theater': 'regal meridian 16',
  'moviename': 'the witch',
  'starttime': '9:30 pm'}}

### Testing State Tracker

In [15]:
class StateTracker:
    """Tracks the state of the episode/conversation and prepares the state representation for the agent."""

    def __init__(self, database, constants):
        """
        The constructor of StateTracker.
        The constructor of StateTracker which creates a DB query object, creates necessary state rep. dicts, etc. and
        calls reset.
        Parameters:
            database (dict): The database with format dict(long: dict)
            constants (dict): Loaded constants in dict
        """

        self.db_helper = DBQuery(database)
        self.match_key = usersim_default_key
        self.intents_dict = convert_list_to_dict(all_intents)
        self.num_intents = len(all_intents)
        self.slots_dict = convert_list_to_dict(all_slots)
        self.num_slots = len(all_slots)
        self.max_round_num = constants['run']['max_round_num']
        self.none_state = np.zeros(self.get_state_size())
        self.reset()

In [16]:
db_helper = DBQuery(database) #loading the database


In [17]:
usersim_default_key = 'ticket'
all_intents = ['inform', 'request', 'done', 'match_found', 'thanks', 'reject']
all_slots = ['actor', 'actress', 'city', 'critic_rating', 'date', 'description', 'distanceconstraints',
             'genre', 'greeting', 'implicit_value', 'movie_series', 'moviename', 'mpaa_rating',
             'numberofpeople', 'numberofkids', 'other', 'price', 'seating', 'starttime', 'state',
             'theater', 'theater_chain', 'video_format', 'zip', 'result', usersim_default_key, 'mc_list']

match_key = usersim_default_key
num_intents = len(all_intents)
slots_dict = convert_list_to_dict(all_slots)
num_slots = len(all_slots)
max_round_num = constants['run']['max_round_num']

#### Getting size of the state function

In [18]:
def get_state_size(num_intents, num_slots, max_round_num):
        """Returns the state size of the state representation used by the agent."""

        return 2 * num_intents + 7 * num_slots + 3 + max_round_num

In [19]:
state_size = get_state_size(num_intents, num_slots, max_round_num)
state_size

224

In [20]:
def get_state_size(self):
    """Returns the state size of the state representation used by the agent."""

    return 2 * self.num_intents + 7 * self.num_slots + 3 + self.max_round_num

#### Update Agent State

In [21]:
history = []
round_num=0

In [51]:
agent_action = {'intent': 'match_found', 'inform_slots': {},
                                 'request_slots': {'theatre':'regal meridian 16'}}

#### Update State Agent
adds agents action to the agent action history and adds action parameters with query information
if the intent for an episode in a conversation is "inform" the it calls function "fillinforslots" to fill the information required by "agent_informs" <br>
if the intent is "match_found" then there should not be any "inform_slots" for information request and the "request_slot" should have a value that reterieves the results from database with respect to the movie ticket results

In [52]:
def update_state_agent(agent_action):
        """
        Updates the dialogue history with the agent's action and augments the agent's action.
        Takes an agent action and updates the history. Also augments the agent_action param with query information and
        any other necessary information.
        Parameters:
            agent_action (dict): The agent action of format dict('intent': string, 'inform_slots': dict,
                                 'request_slots': dict) and changed to dict('intent': '', 'inform_slots': {},
                                 'request_slots': {}, 'round': int, 'speaker': 'Agent')
        """

        if agent_action['intent'] == 'inform':
            assert agent_action['inform_slots']
            inform_slots = db_helper.fill_inform_slot(agent_action['inform_slots'], current_informs)
            agent_action['inform_slots'] = inform_slots
            assert agent_action['inform_slots']
            key, value = list(agent_action['inform_slots'].items())[0]  # Only one
            assert key != 'match_found'
            assert value != 'PLACEHOLDER', 'KEY: {}'.format(key)
            current_informs[key] = value
        # If intent is match_found then fill the action informs with the matches informs (if there is a match)
        elif agent_action['intent'] == 'match_found':
            assert not agent_action['inform_slots'], 'Cannot inform and have intent of match found!'
            db_results = db_helper.get_db_results(current_informs)
            if db_results:
                # Arbitrarily pick the first value of the dict
                key, value = list(db_results.items())[0]
                agent_action['inform_slots'] = copy.deepcopy(value)
                agent_action['inform_slots'][match_key] = str(key)
            else:
                agent_action['inform_slots'][match_key] = 'no match available'
                current_informs[match_key] = agent_action['inform_slots'][match_key]
        agent_action.update({'round': round_num, 'speaker': 'Agent'})
        history.append(agent_action)

In [53]:
updated_state_agent = update_state_agent(agent_action)

In [54]:
history

[{'intent': 'inform',
  'inform_slots': {'moviename': 'zootopia'},
  'request_slots': {'theatre': ''},
  'round': 0,
  'speaker': 'Agent'},
 {'intent': 'inform',
  'inform_slots': {'moviename': 'zootopia'},
  'request_slots': {'theatre': ''},
  'round': 0,
  'speaker': 'Agent'},
 {'intent': 'match_found',
  'inform_slots': {'city': 'seattle',
   'theater': 'regal meridian 16',
   'zip': '98101',
   'numberofkids': 'two',
   'theater_chain': 'regal meridian',
   'state': 'wa',
   'other': 'movie assistant number',
   'starttime': '6:30pm',
   'date': 'tonight',
   'moviename': 'zootopia',
   'ticket': '10'},
  'request_slots': {'theatre': 'regal meridian 16'},
  'round': 0,
  'speaker': 'Agent'}]

#### Update State User
This function appends user information passed into the history

In [65]:
user_action = {'intent': 'request', 'inform_slots': {'moviename':'the witch','city':'seattle'},
                                 'request_slots': {'theatre':''}}

In [75]:
history =[]


In [86]:
def update_state_user(user_action):
        """
        Updates the dialogue history with the user's action and augments the user's action.
        Takes a user action and updates the history. Also augments the user_action param with necessary information.
        Parameters:
            user_action (dict): The user action of format dict('intent': string, 'inform_slots': dict,
                                 'request_slots': dict) and changed to dict('intent': '', 'inform_slots': {},
                                 'request_slots': {}, 'round': int, 'speaker': 'User')
        """
        for key, value in user_action['inform_slots'].items():
                current_informs[key] = value
        user_action.update({'round': round_num, 'speaker': 'User'})
        history.append(user_action)

In [87]:
updated_state_user = update_state_user(user_action)

In [88]:
history

[{'intent': 'request',
  'inform_slots': {'moviename': 'the witch', 'city': 'seattle'},
  'request_slots': {'theatre': ''},
  'round': 0,
  'speaker': 'User'},
 {'intent': 'request',
  'inform_slots': {'moviename': 'the witch', 'city': 'seattle'},
  'request_slots': {'theatre': ''},
  'round': 0,
  'speaker': 'User'},
 {'intent': 'request',
  'inform_slots': {'moviename': 'the witch', 'city': 'seattle'},
  'request_slots': {'theatre': ''},
  'round': 0,
  'speaker': 'User'}]

#### Get State

Returns current state of conversation for agent to be consumed into the Q network in the form -> user_act_rep, user_inform_slots_rep, user_request_slots_rep, agent_act_rep, agent_inform_slots_rep,
             agent_request_slots_rep, current_slots_rep, turn_rep, turn_onehot_rep, kb_binary_rep,
             kb_count_rep converted into numpy array of shape (state size,) 

In [141]:
history = [{'intent': 'inform',
  'inform_slots': {'moviename': 'the witch'},
  'request_slots': {'genre':'thriller'},
  'round': 0,
  'speaker': 'Agent'},{'intent': 'request',
  'inform_slots': {'moviename': 'the witch', 'city': 'seattle'},
  'request_slots': {'genre':''},
  'round': 0,
  'speaker': 'User'}]

In [142]:
db_helper = DBQuery(database)
match_key = usersim_default_key
intents_dict = convert_list_to_dict(all_intents)
num_intents = len(all_intents)
slots_dict = convert_list_to_dict(all_slots)
num_slots = len(all_slots)
max_round_num = constants['run']['max_round_num']


In [143]:
def get_state(done=False):
        """
        Returns the state representation as a numpy array which is fed into the agent's neural network.
        The state representation contains useful information for the agent about the current state of the conversation.
        Processes by the agent to be fed into the neural network. Ripe for experimentation and optimization.
        Parameters:
            done (bool): Indicates whether this is the last dialogue in the episode/conversation. Default: False
        Returns:
            numpy.array: A numpy array of shape (state size,)
        """

        # If done then fill state with zeros
#         if done:
#             return self.none_state

        user_action = history[-1]
        db_results_dict = db_helper.get_db_results_for_slots(current_informs)
        last_agent_action = history[-2] if len(history) > 1 else None

        # Create one-hot of intents to represent the current user action
        user_act_rep = np.zeros((num_intents,))
        user_act_rep[intents_dict[user_action['intent']]] = 1.0

        # Create bag of inform slots representation to represent the current user action
        user_inform_slots_rep = np.zeros((num_slots,))
        for key in user_action['inform_slots'].keys():
            user_inform_slots_rep[slots_dict[key]] = 1.0

        # Create bag of request slots representation to represent the current user action
        user_request_slots_rep = np.zeros((num_slots,))
        print("keys user requests : ", user_action)
        for key in user_action['request_slots'].keys():
            user_request_slots_rep[slots_dict[key]] = 1.0

        # Create bag of filled_in slots based on the current_slots
        current_slots_rep = np.zeros((num_slots,))
        for key in current_informs:
            current_slots_rep[slots_dict[key]] = 1.0

        # Encode last agent intent
        agent_act_rep = np.zeros((num_intents,))
        if last_agent_action:
            agent_act_rep[intents_dict[last_agent_action['intent']]] = 1.0

        # Encode last agent inform slots
        agent_inform_slots_rep = np.zeros((num_slots,))
        if last_agent_action:
            for key in last_agent_action['inform_slots'].keys():
                agent_inform_slots_rep[slots_dict[key]] = 1.0

        # Encode last agent request slots
        agent_request_slots_rep = np.zeros((num_slots,))
        if last_agent_action:
            for key in last_agent_action['request_slots'].keys():
                agent_request_slots_rep[slots_dict[key]] = 1.0

        # Value representation of the round num
        turn_rep = np.zeros((1,)) + round_num / 5.

        # One-hot representation of the round num
        turn_onehot_rep = np.zeros((max_round_num,))
        turn_onehot_rep[round_num - 1] = 1.0

        # Representation of DB query results (scaled counts)
        kb_count_rep = np.zeros((num_slots + 1,)) + db_results_dict['matching_all_constraints'] / 100.
        for key in db_results_dict.keys():
            if key in slots_dict:
                kb_count_rep[slots_dict[key]] = db_results_dict[key] / 100.

        # Representation of DB query results (binary)
        kb_binary_rep = np.zeros((num_slots + 1,)) + np.sum(db_results_dict['matching_all_constraints'] > 0.)
        for key in db_results_dict.keys():
            if key in slots_dict:
                kb_binary_rep[slots_dict[key]] = np.sum(db_results_dict[key] > 0.)

        state_representation = np.hstack(
            [user_act_rep, user_inform_slots_rep, user_request_slots_rep, agent_act_rep, agent_inform_slots_rep,
             agent_request_slots_rep, current_slots_rep, turn_rep, turn_onehot_rep, kb_binary_rep,
             kb_count_rep]).flatten()

        return state_representation

In [144]:
x = get_state()

keys user requests :  {'intent': 'request', 'inform_slots': {'moviename': 'the witch', 'city': 'seattle'}, 'request_slots': {'genre': ''}, 'round': 0, 'speaker': 'User'}


In [145]:
x

array([0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.